In [ ]:
import tensorflow as tf
import numpy as np
import csv
import pandas as pd
import numpy as np
from scipy.ndimage.interpolation import shift

In [ ]:
# Parameters
RANDOM_SEED = 42
tf.set_random_seed(RANDOM_SEED)

days_history = 4
features = 4
learningrate = 0.02
epochs = 100
percent_training = 0.8
mid_layer_size = 32


In [ ]:
# Get dataset

def get_dataset(name):
    df = pd.read_csv(name, sep=';', index_col=0, encoding="Latin1")
    df = df[::-1]
    df.columns = ['Siste', 'Kjoper', 'Selger', 'Hoy', 'Lav', 'Totalt_omsatt_NOK', 'Totalt_antall_aksjer_omsatt', 'Antall_off_handler', 'Antall_handler_totalt', 'VWAP']
    return df


def get_currency_data(name):
    df = pd.read_csv(name, sep=';', index_col=0)
    return df


def append_currency_to_stock_price_df(df_stock, df_currency):
    df_stock["Dollarkurs"] = df_currency["1 USD"]


MHG = get_dataset("data\\MHG_data.csv")
nok = get_currency_data("data\\Valutakurser.csv")

append_currency_to_stock_price_df(MHG, nok)

In [ ]:
# Initialize labels

Up = MHG.Siste.diff(1) > 0

n = len(Up)

train = np.zeros((n, 2))
train[:,0] = (Up.values == 0)
train[:,1] = (Up.values == 1)

cutoff = int(np.floor(n*percent_training))

Y_train = train[days_history:cutoff,:]
Y_test = train[cutoff:,:]

In [ ]:
# Initialize training data
input_dimension = features*days_history

Siste_all = np.zeros((n, days_history))
Siste = MHG.Siste.values

Omsatte_all = np.zeros((n, days_history))
Omsatte = MHG.Totalt_antall_aksjer_omsatt.values

Antall_handler_all = np.zeros((n, days_history))
Antall_handler = MHG.Antall_handler_totalt.values

Valutakurs_all = np.zeros((n, days_history))
Valutakurs = MHG.Dollarkurs.values

for i in range(days_history):
    #print(i)
    Siste_all[:,i] = shift(Siste, i, cval= np.NaN)
    Omsatte_all[:,i] = shift(Omsatte, i, cval= np.NaN)
    Antall_handler_all[:,i] = shift(Antall_handler, i, cval= np.NaN)
    Valutakurs_all[:,i] = shift(Valutakurs, i, cval= np.NaN)

Siste_all = Siste_all - min(Siste)
Omsatte_all = Omsatte_all - min(Omsatte)
Antall_handler_all = Antall_handler_all - min(Antall_handler)
Valutakurs_all = Valutakurs_all - min(Valutakurs)	

Siste_all = Siste_all / (max(Siste) - min(Siste))
Omsatte_all = Omsatte_all / (max(Omsatte) - min(Omsatte))
Antall_handler_all = Antall_handler_all / (max(Antall_handler) - min(Antall_handler))
Valutakurs_all = Valutakurs_all/ (max(Valutakurs) - min(Valutakurs))


X = np.zeros((n, input_dimension))
X[:,0*days_history:(0+1)*days_history] = Siste_all
X[:,1*days_history:(1+1)*days_history] = Omsatte_all
X[:,2*days_history:(2+1)*days_history] = Antall_handler_all
X[:,3*days_history:(3+1)*days_history] = Valutakurs_all
print(X.shape)

X_train = X[days_history:cutoff,:]
X_test = X[cutoff:,:]

In [ ]:
# Setup graph
def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=1)
    return tf.Variable(weights)


# Layer's sizes
x_size = X_train.shape[1]  
h_size = mid_layer_size              
y_size = Y_train.shape[1]   


# Symbols
X = tf.placeholder("float", shape=[None, x_size])
y = tf.placeholder("float", shape=[None, y_size])

# Weight initializations
w_1 = init_weights((x_size, h_size))
w_2 = init_weights((h_size, y_size))

# Forward propagation

h    = tf.nn.tanh(tf.matmul(X, w_1)) 
yhat = tf.nn.softmax(tf.matmul(h, w_2))
predict = tf.argmax(yhat, axis=1)

# Backward propagation

cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
updates = tf.train.GradientDescentOptimizer(learningrate).minimize(cost)


In [ ]:
# Run graph
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for epoch in range(epochs):
    for i in range(len(X_train)):
        sess.run(updates, feed_dict={X: X_train[i: i + 1], y: Y_train[i: i + 1]})

    train_accuracy = np.mean(np.argmax(Y_train, axis=1) ==
                 sess.run(predict, feed_dict={X: X_train, y: Y_train}))
    test_accuracy  = np.mean(np.argmax(Y_test, axis=1) ==
                 sess.run(predict, feed_dict={X: X_test, y: Y_test}))
    number_of_negatives = np.average( sess.run(predict, feed_dict={X: X_train, y: Y_train}) == 0)
    number_of_positives = np.average( sess.run(predict, feed_dict={X: X_train, y: Y_train}) == 1)
    print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%. Positives: %.2f%%. Negatives %.2f%%."
          % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy, 100. * number_of_positives, 100. * number_of_negatives))


In [ ]:
test_results = (np.argmax(test_y, axis=1) == sess.run(predict, feed_dict={X: test_X, y: test_y}))
print(sess.run(yhat, feed_dict={X: test_X, y: test_y}))
print(sess.run(predict, feed_dict={X: test_X, y: test_y}))
print(np.argmax(test_y, axis=1))




In [ ]:
sess.close()